# Modelo Relacional

Categoria(_id_, nome)

Regiao(_id_, nome)

Grupo(_id_, nome)

Receita(_id_, titulo, fonte, instrucoes, regiao_id, categoria_id)
  regiao_id chave estrangeira -> Regiao(id)
  categoria_id chave estrangeira -> Categoria(id)

Ingrediente(_id_, nome, grupo_id)
  grupo_id chave estrangeira -> Grupo(id)

ReceitaIngrediente(_receita_id_, _ingrediente_id_, qtde)
  _receita_id_ chave estrangeira -> Receita(id)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)

Nutriente(_id_, nome)

IngredienteNutriente(_ingrediente_id_, _nutriente_id_, valor, min, max)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _nutriente_id_ chave estrangeira -> Nutriente(id)

ComponenteQuimico(_id_, nome, formula)

IngredienteComponenteQuimico(_ingrediente_id_, _componenteQuimico_id_)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _componenteQuimico_id_ chave estrangeira -> ComponenteQuimico(id)


In [1]:
%defaultDatasource jdbc:h2:mem:db

UsageError: Line magic function `%defaultDatasource` not found.


In [5]:
%sql CREATE TABLE Categoria (
  id_ INTEGER PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_categories.csv');


%sql CREATE TABLE Regiao (
  id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_regions.csv');


%sql CREATE TABLE Grupo (
  id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_groups');


%sql CREATE TABLE Receita (
  id_ INT PRIMARY KEY,
  titulo VARCHAR(255) NOT NULL,
  fonte VARCHAR(255),
  instrucoes TEXT,
  regiao_id INT,
  categoria_id INT,
  FOREIGN KEY (regiao_id) REFERENCES Regiao(_id_),
  FOREIGN KEY (categoria_id) REFERENCES Categoria(_id_)
) AS SELECT
  _id_, titulo, fonte, instrucoes, regiao_id, categoria_id
FROM CSVREAD('../../data/processed/p_recipes.csv');


%sql CREATE TABLE Ingrediente (
  id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  grupo_id INT,
  FOREIGN KEY (grupo_id) REFERENCES Grupo(_id_)
) AS SELECT
  id_, nome, grupo_id
FROM CSVREAD('../../data/processed/p_ingredients.csv');


%sql CREATE TABLE ReceitaIngrediente (
  receita_id_ INT,
  ingrediente_id_ INT,
  qtde INT,
  PRIMARY KEY (_receita_id_, _ingrediente_id_),
  FOREIGN KEY (_receita_id_) REFERENCES Receita(_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_)
) AS SELECT
  _receita_id, _ingrediente_id_, qtde
FROM CSVREAD('../../data/processed/p_ingredients_recipes.csv');

%sql CREATE TABLE Nutriente (
  id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_nutrients.csv');


%sql CREATE TABLE IngredienteNutriente (
  ingrediente_id_ INT,
  nutriente_id_ INT,
  valor DOUBLE,
  min DOUBLE,
  max DOUBLE,
  PRIMARY KEY (_ingrediente_id_, _nutriente_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_),
  FOREIGN KEY (_nutriente_id_) REFERENCES Nutriente(_id_)
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_ingredients_nutrients.csv');


%sql CREATE TABLE ComponenteQuimico (
  id_ INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  formula VARCHAR(255)
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_compounds.csv');


%sql CREATE TABLE IngredienteComponenteQuimico (
  ingrediente_id_ INT,
  componenteQuimico_id_ INT,
  PRIMARY KEY (_ingrediente_id_, _componenteQuimico_id_),
  FOREIGN KEY (_ingrediente_id_) REFERENCES Ingrediente(_id_),
  FOREIGN KEY (_componenteQuimico_id_) REFERENCES ComponenteQuimico(_id_)
) AS SELECT
  id_, nome
FROM CSVREAD('../../data/processed/p_ingredients_compounds.csv');

IndentationError: unexpected indent (1140945666.py, line 2)

In [ ]:
1 - Quais os ingredientes que compõem determinadas receitas?
SELECT r.titulo AS receita, i.nome AS ingrediente, ri.qtde
FROM ReceitaIngrediente ri
JOIN Receita r ON ri._receita_id_ = r._id_
JOIN Ingrediente i ON ri._ingrediente_id_ = i._id_
WHERE r.titulo = 'NomeDaReceita';

In [6]:
2 - Quais os ingredientes mais consumidos e em cada região?
SELECT r.regiao_id, i.nome AS ingrediente, SUM(ri.qtde) AS quantidade_consumida
FROM ReceitaIngrediente ri
JOIN Ingrediente i ON ri._ingrediente_id_ = i._id_
JOIN Receita r ON ri._receita_id_ = r._id_
GROUP BY r.regiao_id, i.nome
ORDER BY quantidade_consumida DESC;

In [ ]:
3 - Quais os nutrientes mais consumidos em cada região?
SELECT r.regiao_id, n.nome AS nutriente, SUM(in.valor) AS quantidade_consumida
FROM IngredienteNutriente in
JOIN Nutriente n ON in._nutriente_id_ = n._id_
JOIN Ingrediente i ON in._ingrediente_id_ = i._id_
JOIN ReceitaIngrediente ri ON i._id_ = ri._ingrediente_id_
JOIN Receita r ON ri._receita_id_ = r._id_
GROUP BY r.regiao_id, n.nome
ORDER BY quantidade_consumida DESC;

In [ ]:
4 - Dado as receitas de determinada categoria, quais são os componentes (químicos) que menos aparecem e quais são os que mais aparecem?
WITH ComponentesPorReceita AS (
    SELECT r._id_ AS receita_id, cq.nome AS componente
    FROM Receita r
    JOIN ReceitaIngrediente ri ON r._id_ = ri._receita_id_
    JOIN Ingrediente i ON ri._ingrediente_id_ = i._id_
    JOIN IngredienteComponenteQuimico icq ON i._id_ = icq._ingrediente_id_
    JOIN ComponenteQuimico cq ON icq._componenteQuimico_id_ = cq._id_
    WHERE r.categoria_id = 'sua_categoria_id'
)
SELECT componente, COUNT(receita_id) AS quantidade
FROM ComponentesPorReceita
GROUP BY componente
ORDER BY quantidade DESC;